In [336]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [337]:
cd ~/demres

/Users/zurfarosa/demres


In [338]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
import pylab as pl
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.common import codelists
from demres.common.helper_functions import *

In [339]:
%matplotlib inline

In [340]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [341]:
# pt_features = pt_features.drop(['matchid','patid','data_start','data_end','final dementia medcode','index_date'],axis=1)

In [342]:
# pt_features.pracid = pt_features.pracid.astype(str)

In [343]:
pt_features.sort_values(by='isCase')

,patid,gender,yob,pracid,index_date,isCase,final dementia medcode,data_start,data_end,matchid,...,sga_depots_pdds_window0,fga_depots_pdds_window0,antidepressants_pdds_window0,mood_stabilisers_pdds_window1,sedatives_pdds_window1,fgas_pdds_window1,sgas_pdds_window1,sga_depots_pdds_window1,fga_depots_pdds_window1,antidepressants_pdds_window1
6367,17949088,2,1923,88,2006-10-11,False,0.0,1996-06-08,2011-02-14,11850.0,...,0.0,0.0,0.00,0.00,128.29,0.00,0.0,0.0,0.0,0.00
9597,25677470,1,1923,470,2007-10-23,False,0.0,1997-01-28,2013-04-23,70471.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00
9598,8463379,2,1921,379,2009-02-23,False,0.0,1996-03-05,2012-05-29,56368.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00
5099,1780620,1,1928,620,2009-12-07,False,0.0,1999-03-10,2013-04-17,86947.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00
5098,9316371,2,1937,371,2011-03-16,False,0.0,1999-03-12,2013-04-19,55274.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00
5097,953409,2,1922,409,2010-06-25,False,0.0,1998-09-16,2012-10-22,60754.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00
9599,5640242,2,1933,242,2010-04-01,False,0.0,1997-01-28,2013-04-24,36733.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00
5095,7852371,2,1921,371,2009-05-07,False,0.0,1999-03-12,2013-04-17,55265.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00
5094,19915391,1,1931,391,2011-01-19,False,0.0,1999-03-18,2013-04-23,57882.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00
5093,3297565,2,1916,565,2010-01-01,False,0.0,1999-03-15,2013-04-19,81923.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00


In [344]:
pt_features = pd.concat([pt_features[['isCase']],pt_features.filter(regex=("[a-zA-Z0-9]+_window0"))])

In [345]:
pt_features['isCase'].fillna(value=False,inplace=True)

In [346]:
# pt_features = pt_features.loc[:,['isCase','gender','yob','insom_count_window0','consultation_count_window0','insom_count_window1','consultation_count_window1']]

In [347]:
pt_features.head(3)

,antidepressants_pdds_window0,consultation_count_window0,fga_depots_pdds_window0,fgas_pdds_window0,insom_count_window0,isCase,mood_stabilisers_pdds_window0,sedatives_pdds_window0,sga_depots_pdds_window0,sgas_pdds_window0
0,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN


In [348]:
pt_features.describe()

,antidepressants_pdds_window0,consultation_count_window0,fga_depots_pdds_window0,fgas_pdds_window0,insom_count_window0,mood_stabilisers_pdds_window0,sedatives_pdds_window0,sga_depots_pdds_window0,sgas_pdds_window0
count,12736.000000,12736.000000,12736.000000,12736.000000,12736.000000,12736.000000,12736.000000,12736.000000,12736.000000
mean,125.240150,120.308182,0.052057,30.911194,0.223540,45.605292,104.408397,0.000079,2.835819
std,603.080419,83.103290,3.405393,325.675365,0.801774,532.551740,571.176915,0.008861,78.616026
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,63.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,107.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,161.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,13385.400000,1180.000000,271.000000,11638.710000,28.000000,17905.920000,15605.950000,1.000000,5438.560000


In [349]:
# pd.crosstab(pt_features['isCase'],pt_features['consultation_count_window1'])

In [350]:
# pt_features.hist()

In [351]:
train_cols = pt_features.loc[:,pt_features.columns!='isCase']

In [352]:
train_cols

,antidepressants_pdds_window0,consultation_count_window0,fga_depots_pdds_window0,fgas_pdds_window0,insom_count_window0,mood_stabilisers_pdds_window0,sedatives_pdds_window0,sga_depots_pdds_window0,sgas_pdds_window0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [353]:
pt_features['intercept'] = 1.0

In [354]:
pt_features= pt_features.fillna(value=0)

In [356]:
pt_features

,antidepressants_pdds_window0,consultation_count_window0,fga_depots_pdds_window0,fgas_pdds_window0,insom_count_window0,isCase,mood_stabilisers_pdds_window0,sedatives_pdds_window0,sga_depots_pdds_window0,sgas_pdds_window0,intercept
0,0.00,0.0,0.0,0.0,0.0,False,0.00,0.00,0.0,0.00,1.0
1,0.00,0.0,0.0,0.0,0.0,False,0.00,0.00,0.0,0.00,1.0
2,0.00,0.0,0.0,0.0,0.0,False,0.00,0.00,0.0,0.00,1.0
3,0.00,0.0,0.0,0.0,0.0,False,0.00,0.00,0.0,0.00,1.0
4,0.00,0.0,0.0,0.0,0.0,False,0.00,0.00,0.0,0.00,1.0
5,0.00,0.0,0.0,0.0,0.0,False,0.00,0.00,0.0,0.00,1.0
6,0.00,0.0,0.0,0.0,0.0,False,0.00,0.00,0.0,0.00,1.0
7,0.00,0.0,0.0,0.0,0.0,False,0.00,0.00,0.0,0.00,1.0
8,0.00,0.0,0.0,0.0,0.0,False,0.00,0.00,0.0,0.00,1.0
9,0.00,0.0,0.0,0.0,0.0,False,0.00,0.00,0.0,0.00,1.0


In [357]:
logit = sm.Logit(pt_features['isCase'], pt_features[train_cols])

ValueError: Must pass DataFrame with boolean values only

In [358]:
result = logit.fit()

NameError: name 'logit' is not defined

In [332]:
result.summary()

NameError: name 'result' is not defined

In [323]:
np.exp(result.params)

NameError: name 'result' is not defined